In [31]:
import open3d as o3d
import numpy as np
import copy
import time
import plotly.graph_objects as go

np.random.seed(42)

def manual_center(pcd):
    current_center = pcd.get_center()
    translation_to_origin = -current_center
    pcd.translate(translation_to_origin)
    return pcd

# Функция регистрации
def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)
    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    radius_feature = voxel_size * 5
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, 
        [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 0.999))
    return result

def refine_registration(source, target, init_trans, voxel_size):
    distance_threshold = voxel_size * 0.4
    if not target.has_normals():
        target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, init_trans,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result

# Визуализация plotly 
def plot_in_plotly(scene_pcd, source_pcd, transformation, title):
    source_temp = copy.deepcopy(source_pcd)
    source_temp.transform(transformation)

    scene_pts = np.asarray(scene_pcd.points)
    if len(scene_pts) > 15000: scene_pts = scene_pts[::5] # Берем каждую 5-ю точку
    
    src_pts = np.asarray(source_temp.points)
    
    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=scene_pts[:,0], y=scene_pts[:,1], z=scene_pts[:,2],
                mode='markers',
                marker=dict(size=1.5, color='lightgray', opacity=0.3),
                name='Сцена'
            ),
            # Найденный объект - красный 
            go.Scatter3d(
                x=src_pts[:,0], y=src_pts[:,1], z=src_pts[:,2],
                mode='markers',
                marker=dict(size=3, color='red'),
                name='Найденный объект'
            )
        ],
        layout=dict(
            title_text=title,
            scene=dict(aspectmode='data'), # Чтобы не сплющивало!
            margin=dict(l=0, r=0, b=0, t=40)
        )
    )
    fig.show()

In [32]:
def generate_task2_assets():
    # Снеговик 
    s_bottom = o3d.geometry.TriangleMesh.create_sphere(radius=1.0).sample_points_uniformly(1500)
    s_middle = o3d.geometry.TriangleMesh.create_sphere(radius=0.7).sample_points_uniformly(1000).translate((0, 0, 1.2))
    s_top = o3d.geometry.TriangleMesh.create_sphere(radius=0.4).sample_points_uniformly(800).translate((0, 0, 2.0))
    
    nose = o3d.geometry.TriangleMesh.create_cone(radius=0.1, height=0.4).sample_points_uniformly(300)
    R = nose.get_rotation_matrix_from_xyz((-np.pi / 2, 2, 2))
    nose.rotate(R, center=(0, 0, 0)).translate((0.4, 0, 2.0)) 
    snowman = s_bottom + s_middle + s_top + nose

    # Стол 
    table_w, table_h, table_t = 3.0, 2.0, 0.2
    leg_h, leg_r, leg_inset = 1.5, 0.1, 0.2
    
    top = o3d.geometry.TriangleMesh.create_box(table_w, table_h, table_t).sample_points_uniformly(2000)
    top = manual_center(top).translate((0, 0, leg_h + table_t / 2))
    
    leg_tmpl = o3d.geometry.TriangleMesh.create_cylinder(leg_r, leg_h).sample_points_uniformly(500)
    leg_tmpl = manual_center(leg_tmpl)
    
    dx, dy = table_w/2 - leg_inset, table_h/2 - leg_inset
    lz = leg_h / 2
    table = top + \
            copy.deepcopy(leg_tmpl).translate(( dx,  dy, lz)) + \
            copy.deepcopy(leg_tmpl).translate((-dx,  dy, lz)) + \
            copy.deepcopy(leg_tmpl).translate(( dx, -dy, lz)) + \
            copy.deepcopy(leg_tmpl).translate((-dx, -dy, lz))

    # Пирамида 
    cube_s = 1.0
    base_c = o3d.geometry.TriangleMesh.create_box(cube_s, cube_s, cube_s).sample_points_uniformly(500)
    base_c = manual_center(base_c)
    
    pyramid = o3d.geometry.PointCloud()
    # Слой 1
    for i in range(3):
        for j in range(3):
            pyramid += copy.deepcopy(base_c).translate(((i-1)*cube_s, (j-1)*cube_s, cube_s/2))
    # Слой 2
    for i in range(2):
        for j in range(2):
            pyramid += copy.deepcopy(base_c).translate(((i-0.5)*cube_s, (j-0.5)*cube_s, cube_s*1.5))
    # Слой 3
    pyramid += copy.deepcopy(base_c).translate((0, 0, cube_s*2.5))

    # Звезда (Прямоугольники) 
    beam = o3d.geometry.TriangleMesh.create_box(3, 0.5, 0.5).sample_points_uniformly(1000)
    beam = manual_center(beam).translate((1.5, 0, 0))
    star_rect = o3d.geometry.PointCloud()
    for i in range(6):
        R = beam.get_rotation_matrix_from_xyz((0, 0, np.deg2rad(i*60)))
        star_rect += copy.deepcopy(beam).rotate(R, center=(0,0,0))

    # Звезда (Конусы). Я в Task_2 делала для интереса, так и оставила
    cone = o3d.geometry.TriangleMesh.create_cone(0.5, 3.0).sample_points_uniformly(1000)
    R_lay = cone.get_rotation_matrix_from_xyz((0, np.pi/2, 0))
    cone.rotate(R_lay, center=(0,0,0))
    star_cones = o3d.geometry.PointCloud()
    for i in range(6):
        R = cone.get_rotation_matrix_from_xyz((0, 0, np.deg2rad(i*60)))
        star_cones += copy.deepcopy(cone).rotate(R, center=(0,0,0))

    return snowman, table, pyramid, star_rect, star_cones

# Генерируем объекты
snowman, table, pyramid, star_rect, star_cones = generate_task2_assets()

In [33]:
import os

# Папка для результатов
os.makedirs("results", exist_ok=True)

# Файл для записи метрик
metrics_file = open("results/metrics.txt", "w", encoding="utf-8")

# Поиск и сохранение

# Подготовка шаблонов 
tmpl_star_rect = copy.deepcopy(star_rect)
tmpl_star_rect.scale(0.5, center=tmpl_star_rect.get_center())
tmpl_star_cones = copy.deepcopy(star_cones)
tmpl_star_cones.scale(0.5, center=tmpl_star_cones.get_center())

sources = {
    "Снеговик": snowman,
    "Стол": table,
    "Пирамида": pyramid,
    "Звезда из прямоугольников": tmpl_star_rect,
    "Звезда из конусов": tmpl_star_cones
}

VOXEL_SIZE = 0.1
print("Подготовка сцены")
target_down, target_fpfh = preprocess_point_cloud(final_scene, VOXEL_SIZE)

for name, source_orig in sources.items():
    print(f"\n======== Поиск объекта: {name} ========")
    metrics_file.write(f"\n======== Объект: {name} ========\n")
    
    # Центрируем шаблон
    source = copy.deepcopy(source_orig)
    source = manual_center(source)
    
    # RANSAC
    source_down, source_fpfh = preprocess_point_cloud(source, VOXEL_SIZE)
    
    start_time = time.time()
    ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        VOXEL_SIZE * 1.5,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        4, 
        [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(VOXEL_SIZE * 1.5)],
        o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 0.999))
    
    ransac_info = f"Global RANSAC: Fitness={ransac.fitness:.4f}, RMSE={ransac.inlier_rmse:.4f} ({time.time()-start_time:.2f}s)"
    print(f"--> {ransac_info}")
    metrics_file.write(f"{ransac_info}\n")
    
    # ICP
    icp = refine_registration(source, final_scene, ransac.transformation, VOXEL_SIZE)
    
    icp_info = f"ICP Refinement: Fitness={icp.fitness:.4f}, RMSE={icp.inlier_rmse:.4f}"
    print(f"--> {icp_info}")
    metrics_file.write(f"{icp_info}\n")
    metrics_file.write(f"Transformation Matrix:\n{icp.transformation}\n")
    
    # Визуализация и сохранение
    source_temp = copy.deepcopy(source)
    source_temp.transform(icp.transformation)
    
    scene_pts = np.asarray(final_scene.points)
    if len(scene_pts) > 15000: scene_pts = scene_pts[::5]
    src_pts = np.asarray(source_temp.points)
    
    fig = go.Figure(
        data=[
            go.Scatter3d(x=scene_pts[:,0], y=scene_pts[:,1], z=scene_pts[:,2],
                         mode='markers', marker=dict(size=1.5, color='lightgray', opacity=0.3), name='Сцена'),
            go.Scatter3d(x=src_pts[:,0], y=src_pts[:,1], z=src_pts[:,2],
                         mode='markers', marker=dict(size=3, color='red'), name='Найденный объект')
        ],
        layout=dict(title_text=f"Результат: {name}", scene=dict(aspectmode='data'), margin=dict(l=0, r=0, b=0, t=40))
    )
    
    fig.show()
    
    # Сохраняем в файл
    fig.write_html(f"results/{name}.html")
    print(f"Saved: results/{name}.html")

metrics_file.close()
print("\nВсе результаты сохранены в папку 'results'")

Подготовка сцены

======== Поиск объекта: Снеговик ========
--> Global RANSAC: Fitness=1.0000, RMSE=0.0592 (0.14s)
--> ICP Refinement: Fitness=0.6911, RMSE=0.0252


Saved: results/Снеговик.html

======== Поиск объекта: Стол ========
--> Global RANSAC: Fitness=1.0000, RMSE=0.0561 (0.48s)
--> ICP Refinement: Fitness=0.7157, RMSE=0.0254


Saved: results/Стол.html

======== Поиск объекта: Пирамида ========
--> Global RANSAC: Fitness=0.9992, RMSE=0.0693 (1.30s)
--> ICP Refinement: Fitness=0.4697, RMSE=0.0279


Saved: results/Пирамида.html

======== Поиск объекта: Звезда из прямоугольников ========
--> Global RANSAC: Fitness=1.0000, RMSE=0.0468 (0.22s)
--> ICP Refinement: Fitness=0.9632, RMSE=0.0216


Saved: results/Звезда из прямоугольников.html

======== Поиск объекта: Звезда из конусов ========
--> Global RANSAC: Fitness=1.0000, RMSE=0.0449 (0.16s)
--> ICP Refinement: Fitness=0.9778, RMSE=0.0201


Saved: results/Звезда из конусов.html

Все результаты сохранены в папку 'results'
